In [8]:
import pandas as pd
from datetime import date
import pprint as pp

## Time Slots

In [9]:
# Time Slots
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
n_days = len(days)
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']
n_times = len(times)
lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len:int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s,e))
    return pairs
# No lectures start/end in these time slots (as (day, time) tuples) --- labs, tutorials are OK
mon_2pm = (days.index('Mon'), times.index('14:00'),)
wed_2pm = (days.index('Wed'), times.index('14:00'),)
no_lecture_slots = ([(mon_2pm[0], t,) for t in range(mon_2pm[1], (mon_2pm[1] + 6))] +
                    [(wed_2pm[0], t,) for t in range(wed_2pm[1], (wed_2pm[1] + 6))])

def overlap(b:int, e:int, t:int) -> bool:
    # Returns a bool indicating whether the intervel (b,e) contains time t
    # b,e,t are indexes into the 'times' array
    assert (0 <= b < n_times) and(0 <= e < n_times) and (0 <= t < n_times)
    return (b <= t <= e)



## Classrooms

In [10]:
# classroom_cols = {'RoomNum': str, 'Capacity': int, 'Category': str,
#                   'HasProjector': bool, 'HasWacom': bool}
classroom_cols = {'RoomNumber': str, 'Capacity': int}
classrooms = pd.read_csv('classroom_master.csv', dtype=classroom_cols)
n_rooms = classrooms.shape[0]

#get room capacity given room id 
def get_room_capacity(room_id: int) -> int:
    return int(classrooms.loc[room_id, 'Capacity'])

def get_room_category(room_id: int) -> str:
    a=classrooms.loc[room_id, 'Lab']
    b=classrooms.loc[room_id, 'DeptLab']
    if a:
        if b:
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_room_num(room_id: int) -> str:
    return classrooms.loc[room_id, 'RoomNumber']

def check_cse_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'CSE Lab':
        return True
    return False

def check_ece_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'ECE Lab':
        return True
    return False


def capacity_room_type(room_type: str) -> int:
    if room_type == 'CSE Lab':
        count=0
        for i in range(n_rooms):
            if check_cse_lab_room(i):
                count+=get_room_capacity(i)
        return count
    elif room_type == 'ECE Lab':
        count=0
        for i in range(n_rooms):
            if check_ece_lab_room(i):
                count+=get_room_capacity(i)
        return count
    else:
        count=0
        for i in range(n_rooms):
            if not check_cse_lab_room(i) and not check_ece_lab_room(i):
                count+=get_room_capacity(i)
        return count
    
print(capacity_room_type('CSE Lab'))
print(capacity_room_type('ECE Lab'))
print(capacity_room_type('NotLab'))

185
65
2165


## Courses & Enrollment

In [11]:
courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()

course_id = {}
for i in range(n_courses):
    course_id[courses['Code'][i]] = i

course_id_code={}
for i in range(n_courses):
    course_id_code[i] = courses['Code'][i]

def get_course_id(course_code: str) -> int:
    return course_id[course_code.strip()]

def get_course_strength(course_id: int) -> int:
    return int(enrollment[enrollment['CourseCode'] == courses['Code'][course_id]].shape[0])

def get_courses_student(student_id: int) -> list:
    l =  enrollment[enrollment['EmailID'] == students[student_id]]['CourseCode'].values
    return [get_course_id(x) for x in l]
    
def get_course_title(course_id: int) -> str:
    return courses.loc[course_id, 'Title']
    

def get_course_category(course_id: int) -> str:
    
    a = courses.loc[course_id,'Lab']
    b = courses.loc[course_id,'Dept']
    if a:
        if b == 'CSE':
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_dept(course_id: int) -> str:
    return courses.loc[course_id, 'Dept']

def get_not_rooms_course(course_id: int) -> list:
    l=[]
    for i in range(n_rooms):
        if get_room_category(i) != get_course_category(course_id):
            l.append(i)
    return l

def get_course_duration(course_id: int) -> int:
    return courses.loc[course_id, 'Exam Duration']


def get_preferred_slots(course_id: int) -> list:
    x = courses.loc[course_id, 'Preferred Timings'].split(',')
    triplets = []
    for i in range(0, len(x), 3):
        begin = x[i].strip(" [(\"'")
        end = x[i+1].strip(" )'\"]")
        day = x[i+2].strip(" )'\"]")
        triplets.append((begin, end, day))
    for i in range(len(triplets)):
        triplets[i] = (times.index(triplets[i][0]), times.index(triplets[i][1]), days.index(triplets[i][2]))
    return triplets

course_enrollment = {} # for a particular course, no.of students are enrolled

for i in range(n_courses):
    # course_enrollment[i] = get_course_strength(i)
    a = get_course_strength(i)
    b = capacity_room_type(get_course_category(i))
    if a<=b:
        course_enrollment[i] = True
    else:
        course_enrollment[i] = False

pp.pprint(course_enrollment)

for i in course_enrollment:
    if course_enrollment[i] == False:
        print(get_course_title(i))
        print(get_course_strength(i))
        print(capacity_room_type(get_course_category(i)))
        print(get_course_category(i))
        print('')
    


{0: True,
 1: True,
 2: True,
 3: True,
 4: True,
 5: True,
 6: True,
 7: True,
 8: True,
 9: True,
 10: True,
 11: True,
 12: True,
 13: True,
 14: True,
 15: True,
 16: True,
 17: True,
 18: True,
 19: True,
 20: True,
 21: True,
 22: True,
 23: True,
 24: True,
 25: True,
 26: True,
 27: True,
 28: True,
 29: True,
 30: True,
 31: True,
 32: True,
 33: True,
 34: True,
 35: True,
 36: True,
 37: True,
 38: True,
 39: True,
 40: True,
 41: True,
 42: True,
 43: True,
 44: True,
 45: True,
 46: True,
 47: True,
 48: True,
 49: True,
 50: True,
 51: True,
 52: True}


In [12]:

from collections import defaultdict

course_student = defaultdict(list)
intersection_pairs = set()
intersection_triplets = set()
for i in range(n_students):
    courses_taken = get_courses_student(i)
    for x in courses_taken:
        course_student[x].append(i)
    for j in range(len(courses_taken)):
        for k in range(j+1, len(courses_taken)):
            intersection_pairs.add((courses_taken[j], courses_taken[k]))
            for l in range(k+1, len(courses_taken)):
                intersection_triplets.add((courses_taken[j], courses_taken[k], courses_taken[l]))
    
intersection_pairs = list(intersection_pairs)
n_intersection_pairs = len(intersection_pairs)
# print(intersection_pairs)
# print(course_student)
# print(intersection_triplets)

In [13]:
def get_intersection_of_courses(c1, c2):
    return (set(course_student[c1]) & set(course_student[c2]))
# print(course_student[4])
# print(course_student[24])
# print(get_intersection_of_courses(24, 4))

## Faculty

In [14]:
faculty_cols = {'Name': str, 'Course': str, 'Start': str, 'End': str,
                'PreferredSlot': str, 'Flexibility': int}
faculty = pd.read_csv('faculty_master.csv', dtype=faculty_cols)